In [1]:
from models import FEAOF
from const import Descriptors
from new_unit import Data, f1_acc, get_config, cross_validate
from tqdm import tqdm
import pandas as pd
from torch_geometric.loader import DataLoader
from typing import List
import torch
import os
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, roc_curve,precision_recall_curve, auc
from math import sqrt
import pickle
# from herg_em import fetch_sequence, cal_em
from unit import compute_metrics,predict

*** Open Babel Error  in openLib
  /home/ubuntu/miniconda3/envs/my_evs/lib/openbabel/3.1.0/png2format.so did not load properly.
 Error: libXrender.so.1: cannot open shared object file: No such file or directory


In [2]:
Moldel_Type = "FEAOF"
Model = "MPNN"
FingerPrint = "2FP"
Des_IF = "11Des_1441IF_0FC"

file_name = f"./trained_models/{Moldel_Type}/{Model}_{FingerPrint}_{Des_IF}.csv"
with open(file_name,"w") as p:
    p.write("auc_score, pr_auc, AC, f1, SN, PR, SP, CCR, MCC\n")

para form gpu to cpu

In [3]:
# import torch
# import pickle

# model_save_path = '/content/0FC_FEAOF.pkl'
# # 加载原始 CUDA 模型参数（因为原本是GPU上保存的）
# with open(model_save_path, 'rb') as handle:
#     model_para = pickle.load(handle)

# # 把所有张量移动到 CPU
# model_para_cpu = {k: v.cpu() for k, v in model_para.items()}

# # 重新保存为 CPU 版
# cpu_model_save_path = 'model_state_cpu.pkl'
# with open(cpu_model_save_path, 'wb') as handle:
#     pickle.dump(model_para_cpu, handle)


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

config_path = "./configures/optimized/train_split/MPNN_GRAPH.yml"
model_save_path = "./trained_models/FEAOF/model_state_cpu.pkl"

In [5]:
with open(model_save_path, 'rb') as handle:
    model_para = pickle.load(handle)

In [6]:
model_para.keys()

dict_keys(['project_node_feats.0.weight', 'project_node_feats.0.bias', 'gnn_layer.bias', 'gnn_layer.nn.0.weight', 'gnn_layer.nn.0.bias', 'gnn_layer.nn.2.weight', 'gnn_layer.nn.2.bias', 'gnn_layer.lin.weight', 'gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0', 'transformer.lin1.weight', 'transformer.lin1.bias', 'transformer.lin2.weight', 'transformer.lin2.bias', 'transformer.pools.0.S', 'transformer.pools.0.mab.fc_q.weight', 'transformer.pools.0.mab.fc_q.bias', 'transformer.pools.0.mab.layer_k.bias', 'transformer.pools.0.mab.layer_k.lin.weight', 'transformer.pools.0.mab.layer_v.bias', 'transformer.pools.0.mab.layer_v.lin.weight', 'transformer.pools.0.mab.fc_o.weight', 'transformer.pools.0.mab.fc_o.bias', 'transformer.pools.1.mab.fc_q.weight', 'transformer.pools.1.mab.fc_q.bias', 'transformer.pools.1.mab.layer_k.weight', 'transformer.pools.1.mab.layer_k.bias', 'transformer.pools.1.mab.layer_v.weight', 'transformer.pools.1.mab.layer_v.bias', 'transformer.pools.1.m

In [7]:
hyperparameters = get_config(config_path)
hyperparameters = {k: v.item() if isinstance(v, np.generic) else v for k, v in hyperparameters.items()}
# model = Cat_GCN(**hyperparameters).model
model = FEAOF(**hyperparameters).model
model.load_state_dict(model_para)

/home/ubuntu/miniconda3/envs/my_evs/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.GraphMultisetTransformer' is deprecated, use 'nn.aggr.GraphMultisetTransformer' instead
  warnings.warn(out)


<All keys matched successfully>

In [8]:

# uniprot_ID="Q12809"
# uni_id = fetch_sequence(uniprot_ID)
# herg_em = cal_em(uni_id)[0]
herg_em = None

In [ ]:
Val_All_Features = "./data/IF_Data/Set60_All_Features.pkl"
Val = "./data/IF_Data/5_eval_set_60_IF_filter.csv"
Val = Data(Val)
Val(Descriptors.GRAPH)

In [10]:
y_hat = predict(model,Val.x_test,Val_All_Features,Val.test_Ro5_idx,herg_em,device)
compute_metrics(np.array(Val.y_test),y_hat,file_name)

Binary classification performace metrics:
[[82 13]
 [31 72]]
TP, FN, TN, FP
72, 31, 82, 13


In [ ]:
Val_All_Features = "./data/IF_Data/Set70_All_Features.pkl"
Val = "./data/IF_Data/5_eval_set_70_IF_filter.csv"
Val = Data(Val)
Val(Descriptors.GRAPH)

In [12]:
y_hat = predict(model,Val.x_test,Val_All_Features,Val.test_Ro5_idx,herg_em,device)
compute_metrics(np.array(Val.y_test),y_hat,file_name)

Binary classification performace metrics:
[[138  24]
 [ 50 142]]
TP, FN, TN, FP
142, 50, 138, 24
